In [178]:
import numpy as np
import astropy.constants as const
import astropy.units as u
import os
import shutil

In [217]:
class pyRadmc3d:
    """ Class that setups up disk models to be used by RADMC 
    
    Spatial grid for all models is done in spherical
    
    """
    #-----------#:
    # Constants #
    #-----------#
    au        = const.au.cgs.value       # Astronomical Unit  [cm]
    pc        = const.pc.cgs.value       # Parsec             [cm]
    Msun      = const.M_sun.cgs.value    # Solar Mass         [g]
    Lsun      = const.L_sun.cgs.value    # Solar Luminosity   [erg/s]
    Rsun      = const.R_sun.cgs.value    # Solar Radius       [cm]
    sig_boltz = const.sigma_sb.cgs.value # Stefan-Boltzman    [erg cm−2 s−1 K−4]
    Tsun      = 5780                     # Solar Temperature  [K] 
    pi        = np.pi                    # circle stuff lol   [radians]
    
    def writeWavelengthINPUT(self,lamdaMIN, lambdaMAX, nlam):
        lambda_grid = np.geomspace(lamdaMIN, lambdaMAX, nlam)
        print('Writing wavelength_micron.inp ...')
        with open('wavelength_micron.inp', 'w+') as f:
            f.write('%d\n'%(nlam))
            for value in lambda_grid:
                f.write('%13.6e\n'%(value))
        return
    
    def makeSpatialandDensityGrid(self, GRID_PARAMS, DISK_PARAMS):
        # inputs list of grid parameters
        # inputs list ofdisk parametes
        rin_au, rc_au, rout_au, gamma, H100au, beta, Mdust_Msun = DISK_PARAMS
        nr_inner_cells, nr_outer_cells, rfactor, ntheta_cells_low, ntheta_cells_high, theta_change, theta_min, nphi_cells = GRID_PARAMS
        
        au = self.au
        pi = self.pi
        Msun = self.Msun

        #------------------------#
        # radial cell wall setup #
        #------------------------#
        rin     = rin_au * au     # inner radius [cm]   
        rout    = rout_au * au    # outer radius [cm]
        rchange = rfactor * rin  # radius where grid changes from [cm] 

        # setup array args
        r_grid_params = [                                             
            [rin, rchange, nr_inner_cells + 1, False],       # inner radius args
            [*np.log10([rchange, rout]), nr_outer_cells + 1] # outer radius args   
        ]

        # to avoid duplication, endpoint is set to FALSE where rchange 
        # will not be accounted for in inner grid but recovered in outer

        r_inner   = np.linspace(*r_grid_params[0])     # inner FINE grid style
        r_outer   = np.logspace(*r_grid_params[1])     # outer COARSE grid style
        r_grid    = np.concatenate([r_inner, r_outer]) # merge the two grids

        #----------------------------------#
        # Theta cell wall setup (altitude) #
        #----------------------------------#
        

        theta_grid_params = [
            [theta_min, theta_change, ntheta_cells_high + 1, False], # theta high args
            [theta_change, pi/2, ntheta_cells_low + 1]               # theta low args
        ]

        theta_grid_high     = np.linspace(*theta_grid_params[0])                         # COARSE grid spacing
        theta_grid_low      = np.linspace(*theta_grid_params[1])                         # FINE grid spacing near the mid-plane
        theta_grid_1st_half = np.concatenate([theta_grid_high, theta_grid_low])          # 0 - 90 degress
        theta_grid_2nd_half = np.flip(pi - theta_grid_1st_half, 0)[1:]                   # from 90 to 180 degress
        theta_grid          = np. concatenate([theta_grid_1st_half,theta_grid_2nd_half]) # az

        #---------------------#
        # Phi cell wall setup #
        #---------------------#    
        phi_grid   = np.linspace(0, 2*pi, nphi_cells + 1) # polar angles

        
        #----------------------------#
        # Find centers of cell walls #
        #----------------------------#
        nr     = len(r_grid)
        nphi   = len(phi_grid)
        ntheta = len(theta_grid)

        r_center_grid     = 0.5 * ( r_grid[0:nr-1] + r_grid[1:nr] )
        phi_center_grid   = 0.5 * ( phi_grid[0:nphi-1] + phi_grid[1:nphi] )
        theta_center_grid = 0.5 * ( theta_grid[0:ntheta-1] + theta_grid[1:ntheta] )

        nr_center     = len(r_center_grid)
        nphi_center   = len(phi_center_grid)
        ntheta_center = len(theta_center_grid)
        
        #--------------------------------------#
        # Calculate the dust density for model #
        #--------------------------------------#
        #
        #  --------------------------------------------------------------------------------------------  #
        def coordTransformation(r_sph, theta):
            # convert from spherical to cylindrical coordinates
            # inputs: 
            r_cylindrical = r_sph * np.sin(theta)
            z_cylindrical = r_sph * np.cos(theta)
            return r_cylindrical , z_cylindrical  

        def sigma_norm(Mdust, rin_au, rout_au, rc_au, gamma): 

            # compute the normalization constant linked total dust mass
            exponent = 2. - gamma
            density_norm = (
                Mdust * (gamma - 2.) / (2. * pi * rc_au * rc_au) / 
                (
                    np.exp(-(rout_au / rc_au)**exponent) - 
                    np.exp(-(rin_au / rc_au)**exponent)
                )
            )
            return density_norm   

        def volumeDustDensity(r, z, rc, Mdust, sigma0, H100au, gamma, beta):
            # Calculate the volume dust density
            # inputs: radius [cm], z-component [cm], disk mass [g], gamma "radial
            # profile" index, the scale height at 100 au [cm], beta "flaring
            # parameter" index

            def H_r():
                # Scale height
                # Calculate the scale height at some radius
                return H100au * self.au * ( r / ( 100 * au ) )**beta    

            def Sigma_r():
                # surface density in cgs
                return sigma0*( ( r / rc )**( -gamma ) ) * np.exp( - ( r / rc )**( 2 - gamma ) )

            H = H_r()
            Sigma = Sigma_r()
            rho = (
                Sigma / ( np.sqrt(2 * pi) * H ) * 
                np.exp( - 0.5 * ( z * z / ( H * H ) ) ) 
            )
            return rho    
        #  --------------------------------------------------------------------------------------------  #
        
        rc     = rc_au * au 
        Mdust  = Mdust_Msun * Msun
        sigma0 = sigma_norm(Mdust = Mdust, rin_au = rin_au, rout_au = rout_au, 
                                  rc_au = rc_au, gamma = gamma)
        
        # Check to make sure characteristic radius is w/in inner and outer boundary 
        if rin > rc or rout < rc:
            raise ValueError('If rc < rin or rc > rout : volume density calculation will result in negative number') 
        
        #------------------------#
        # Do density calculation #
        #------------------------#
        
        phi, theta, rho = np.meshgrid(phi_center_grid, theta_center_grid, r_center_grid, indexing = 'ij')
        # coord transform 
        rr_cyn , zz_cyn = coordTransformation(r_sph = rho, theta= theta)

        # get indices where calculation occur outside of disk size, density is to be zero there
        rls_disk = np.where(rr_cyn < rin)
        rgrt_disk = np.where(rr_cyn > rout) 

        rho_dust = volumeDustDensity(r = rr_cyn, z = zz_cyn, rc = rc, Mdust = Mdust, sigma0 = sigma0, 
                                          gamma = gamma, H100au = H100au, beta = beta)

        # get indices where calculation occur outside of disk size, density is to be zero there
        rls_disk = np.where(rr_cyn < rin)
        rgrt_disk = np.where(rr_cyn > rout)
        rho_dust[rls_disk and rgrt_disk] = 0

        rho_dust_grid = rho_dust.flatten()
        
        #-------------------------#
        # Write files for radmc3d #
        #-------------------------#
        print('Writing amr_grid.inp ...')
        with open('amr_grid.inp', 'w+') as f:
            # Write formating section
            f.write('1\n')                              # iformat
            f.write('0\n')                              # AMR grid style  (0=regular grid, no AMR)
            f.write('100\n')                            # Coordinate system
            f.write('0\n')                              # grid info
            f.write('1 1 1\n')                          # include coordinate: r, phi, theta   #
            f.write('{} {} {}\n'.format(nr_center,      # grid  cell info
                                        nphi_center, 
                                        ntheta_center)) 

            # Write grid values on one line
            for r in r_grid:
                f.write('{} '.format(r))
            #
            f.write('\n')
            #
            for theta in theta_grid:
                f.write('{} '.format(theta))
            #
            f.write('\n')
            #
            for phi in phi_grid:
                f.write('{} '.format(phi))
            
                
        
        print('Writing dust_density.inp.....')
        with open('dust_density.inp', 'w') as f:
            f.write('1\n')                     # iformat
            f.write('{}\n'.format(rho_dust_grid.size)) # nrcells
            f.write('1\n')                     # dust speicies

            for rho in rho_dust_grid:
                f.write('{}\n'.format(rho))
                    
                    
    def writeRadmcMAIN(self):
        # write the thermal input file to compute dust temperature of model
        print('Writing radmc3d.inp for thermal calculation...')
        with open('radmc3d.inp', 'w+') as f:
            f.write('nphot = 100000\n')
            f.write('istar_sphere = 1\n')
            f.write('scattering_mode_max = 3\n')
            #f.write('')
            # Write formating section
        return
    
    def writeStarFile(self, wavelength, Fnu_star, rstar):
        
        # do a check to make sure stellar model wavelengths match input wavelengths in microns
        if np.array_equal(wavelength, self.radmcWaveMicron()) is not True:
            raise ValueError('stellar photosphere wavelengths do not match radmc wavelength file')
        
        # if pass write the file    
        with open('stars.inp') as f:
            # File formating
            f.write('2\n') # iformat
            f.write('1 {}'.format(wavelength.size)) # nstars nlam
            f.write('{} 1, 0, 0, 0'.format(rstar)) # rstar mstar xstar, ystar, zstar -> centered

            # write wavelengths then star flux
            for _lambda in wavelength:
                f.write('{}\n'.format(_lambda))
            for Fnu in Fnu_star:
                f.write('{}\n'.format(Fnu_star))
        return   
    
    def writeDustOpacFile(self, logagrain):
        for i in range(len(logagrain)):

            dust_library = '/home/tyler/Desktop/Model Library/Radmc-3d Models/DustKappaScatmat Library/'
            dust_file    = 'dustkapscatmat_DSHARPmix_' + str(logagrain[i]) + '_-1_5_500_181_0.05_20_5.0_True.inp'

            source       = dust_library + dust_file
            destination  = os.getcwd() + '/' + dust_file[:-34]

            shutil.copy2(source, destination)
        
        os.system('ls dustkapscatmat_* > dustopac.txt')
        
        dust_species = np.genfromtxt('dustopac.txt', dtype = str)
        nspecies = len(dust_species)
        
        print('Writing dustopac.inp with {} dust species...'.format(nspecies))
        with open('dustopac.inp', 'w+') as f:
            # Write formating section
            f.write('2\n')                           # iformat
            f.write('{}\n'.format(nspecies))         # number species to be used
            f.write('-----------------------------\n') # Stellar information: centered

            # Write species sections 
            for spec in range(nspecies):
                f.write('inputstyle[{}]\n'.format(spec))
                f.write('iquantum[0]\n')
                f.write('{}\n'.format(dust_species[spec]))
                if spec != nspecies - 1:
                    f.write('-----------------------------\n')
        return
      
    def runRADMC3D(self, command = 0):
        if command == 0:
            print('Execute is off')
            print('Select: mctherm [1], SED [2], spectrum[3], image[4]')
        if command == 1:
            os.system('radmc3d mctherm')
        if command == 2:
            os.system('radmc3d SED')
        if command == 3:
            os.system('radmc3d spectrum')
        if command == 4:
            os.system('radmc3d image')
        
        

In [219]:
# TESTING

# wave
lmin = 0.5
lmax = 3000
nlam = 200

# ppdisk
rin_au = 1
rc_au = 5
rout_au = 20
gamma = 1
H100au = 10
beta = 2
Mdust_Msun = 1e-4

ppdisk_properties = [rin_au, rc_au, rout_au, gamma, H100au, beta, Mdust_Msun]

nr_inner_cells    = 5
nr_outer_cells    = 10
rfactor           = 2
ntheta_cells_low  = 5
ntheta_cells_high = 5
theta_change      = np.pi/8
theta_min         = 0
nphi_cells        = 3

ppdisk_grid = [nr_inner_cells, nr_outer_cells, rfactor, ntheta_cells_low, 
               ntheta_cells_high, theta_change, theta_min, nphi_cells]

# Dust
logagrain = np.array([0.0671, 1.02, 4.0833])


pyRadmc3d().writeRadmcMAIN()
pyRadmc3d().writeWavelengthINPUT(lmin, lmax, nlam)
pyRadmc3d().makeSpatialandDensityGrid(ppdisk_grid, ppdisk_properties)
pyRadmc3d().writeDustOpacFile(logagrain)
#pyRadmc3d().writeStarFile

Writing radmc3d.inp for thermal calculation...
Writing wavelength_micron.inp ...
Writing amr_grid.inp ...
Writing dust_density.inp.....
Writing dustopac.inp with 3 dust species...
